# ✈️ AIviate: Your Intelligent Airline Assistant

**AIviate** is a multimodal airline assistant built using **OpenAI's GPT-4o** and **Anthropic’s Claude** models. It offers a seamless, voice-enabled flight experience with multilingual support and dynamic visuals.

---

## 🔧 Core Features

- 🎫 **Flight Booking**: Books flights by collecting name, destination city, and travel date.
- 💰 **Ticket Pricing**: Instantly fetches return ticket prices for top destinations.
- 🌍 **Vacation Image Generation**: Uses **DALL·E 3** to create ultra-realistic vacation images of the destination city.
- 🎙️ **Voice Input & Output**: 
  - Accepts voice input using **Whisper** (OpenAI).
  - Reads replies aloud using **TTS (Text-to-Speech)**.
- 🌐 **Spanish Translation**: Translates assistant replies to natural Spanish using **Claude**.
- 🧠 **Function Calling**: Automatically uses tools for pricing and booking via OpenAI’s function calling.

---

## 🧠 Models Used

| Functionality           | Model / API               |
|------------------------|---------------------------|
| Main Assistant Logic    | `gpt-4o`, `gpt-4o-mini`    |
| Audio Transcription     | `whisper-1`               |
| Text-to-Speech          | `tts-1`                   |
| Image Generation        | `dall-e-3`                |
| Spanish Translation     | `claude-3-haiku-20240307` |

---

## 💻 Tech Stack

- Python
- Gradio
- OpenAI API
- Anthropic API
- dotenv (`.env` configuration)

---

> Type, speak, or upload your query. Get instant answers, voice replies, translated responses, and destination visuals — all in one smart assistant!


## 1.Importing Libraries

In [1]:
import os 
import json
import base64
from io import BytesIO
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import gradio as gr
from PIL import Image

## 2. Setting up the environment

In [2]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key=os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key NOT FOUND!")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:8]}")
else:
    print("Anthropic API Key NOT FOUND!")

openai = OpenAI(api_key=openai_api_key)
anthropic_client = anthropic.Anthropic(api_key=anthropic_api_key)

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-a


## 3. Defining the System Prompts

In [3]:
system_message = "You are a helpful assistant for an airline called AIviate"
system_message += "Give short, courteous answers, not more than 1 sentence."
system_message += "Always be precise,accurate and to the point. If you don't know the answer, say so in a polite way."

system_prompt_translate = "Please trasnlsate the following message into clear,natural spanish."
system_prompt_translate += "Only output the translation. Do not add any explanation."

system_prompt_translate_claude = "Por favor, traduzca el siguiente mensaje al español, natural y correcto. Solo muestre la traducción."

## 4. Tool : Ticket Price

In [4]:
ticket_prices= {"london":"$799","london":"$799","paris":"$899","tokyo":"$1499","berlin":"$499"}

def get_ticket_price(destination_city):
    city = destination_city.strip().lower()
    return ticket_prices.get(city,"Unkwown")

## 5. Tool : Booking

In [5]:
def make_booking(name,city,date):
    booking_id = f"FLT{str(abs(hash((name,city,date)))%100000).zfill(5)}"
    return {
        "message" : f"Booking confirmed for {name} to {city.title()} on {date}.",
        "booking_reference" : booking_id
    }
    

## 6. Defining the Function Tools (OpenAI)

In [6]:
# Price Tool
price_fucntion = {
    "type":"function",
    "function": {
        "name" : "get_ticket_price",
        "description" : "Get the price of return ticket to the destination city. Use when a customer asks about it",
        "parameters" : {
            "type" : "object",
            "properties" : {
                "destination_city" : {"type" : "string", "Description" : "Destination City Name"},
            },
            "required" : ["destination_city"]
        }
    }
}

#Booking Tool
booking_function = {
    "type" : "function",
    "function": {
        "name" : "make_booking",
        "description" : "Make booking for a customer. You must collect their name, destination city & date of travel before booking",
        "parameters" : {
            "type" : "object",
            "properties" : {
                "name": {"type": "string", "description": "Name of the passenger"},
                "city": {"type": "string", "description": "Destination city"},
                "date": {"type": "string", "description": "Travel date (YYYY-MM-DD)"},
            },
            "required" : ["name","city","date"]
        }
    }
    
}

tools = [price_fucntion,booking_function]

## 7. Destination City Image Creator (OpenAI Dall E)

In [7]:
def artist(city):
    image_response = openai.images.generate(
        model = "dall-e-3",
        prompt = f"An image representing a vacation in city {city}, showing tourists eveything unique about the {city}, in an ultra realsitic way",
        size = "1024x1024",
        n=1,
        response_format = "b64_json",
    )

    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

## 8. Audio Generator (I/O): GPT for MAIN & Claude for Spanish Translation

In [8]:
#Audio for talking (Assiatant output) - GPT
def talker(message):
    try:
        response = openai.audio.speech.create(
            model = "tts-1",
            voice = "onyx",
            input = message
        )

        return (response.content,"audio/mp3")
    except Exception:
        return (b'','audio/mp3')
        
#Translation - GPT input, Claude output
def call_openai_translation(text):
    messages = [{"role":"system","content":system_prompt_translate},{"role":"user","content":text}]
    response = openai.chat.completions.create(model="gpt-4o-mini",messages=messages)

    return response.choices[0].message.content


def call_claude_translation(text):
    resp = anthropic_client.messages.create(
        model = "claude-3-haiku-20240307",
        max_tokens = 256,
        temperature = 0.2,
        system = system_prompt_translate_claude,
        messages = [{"role" : "user","content":text}]
    )

    return resp.content[0].text if resp.content else ' '

## 9. Tool Handler

In [9]:
def handle_tool_call_openai(message):
    tool_call = message.tool_calls[0]
    func = tool_call.function
    fn_name = func.name
    arguments = json.loads(func.arguments)

    if fn_name == 'get_ticket_price':
        city = arguments.get('destination_city')
        price = get_ticket_price(city)
        tool_resp = {"role": "tool",
            "content": json.dumps({"destination_city": city, "price": price}),
            "tool_call_id": tool_call.id
        }
        return tool_resp, city, None

    elif fn_name == 'make_booking':
        name = arguments.get('name')
        city = arguments.get('city')
        date = arguments.get('date')
        out = make_booking(name, city, date)
        tool_resp = {
            "role": "tool",
            "content": json.dumps(out),
            "tool_call_id": tool_call.id
        }
        return tool_resp, city, out["booking_reference"]

    else:
        return {"role": "tool", "content": "{}", "tool_call_id": tool_call.id}, "", None

## 10. Main Function

In [10]:
def full_agent(user_message,chat_history,audio_in):
    if audio_in is not None and (not user_message or user_message.strip() == ""):
        try:
            with open(audio_in,"rb") as audio_file:
                transcript = openai.audio.transcriptions.create(
                    model = "whisper-1",
                    file = audio_file
                )
            user_message = transcript.text
        except Exception as e:
            print("Whisper transcription error: ",str(e))
            user_message = "(Audio could not be transcribed)"
            
    all_messages = [{"role":"system", "content": system_message}]
    all_messages.extend(chat_history)
    all_messages.append({"role": "user", "content": user_message})

    reply = ""
    city = ""
    booking_ref = None
    image_obj = None

    # Using GPT for main responses
    response = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = all_messages,
        tools = tools
    )

    finish_reason = response.choices[0].finish_reason
    msg =response.choices[0].message
    
    if finish_reason == "tool_calls":
        tool_resp,city,booking_ref = handle_tool_call_openai(msg)
        all_messages.append(msg)
        all_messages.append(tool_resp)
        next_response = openai.chat.completions.create(
            model="gpt-4o",
            messages=all_messages
        )

        reply = next_response.choices[0].message.content
        image_obj = artist(city) if city else None

    else:
        reply = msg.content

    # Using Claude for translation ONLY
    translation = call_claude_translation(reply)

    audio_bytes, audio_type = talker(reply) if reply else (b'',"audio/mp3")
    
    chat_history.append({"role": "user", "content": user_message})
    if booking_ref:
        reply = f"{reply} (Your booking reference: {booking_ref})"
        translation = f"{translation} (Referencia de reserva: {booking_ref})"
    chat_history.append({"role": "assistant", "content": reply})    

    return chat_history, translation, audio_bytes, image_obj

## 11. User Interface using Gradio

In [11]:
with gr.Blocks(title="AIviate: Your friendly Airline Assiatant!") as demo:
    gr.Markdown("# ✈️ AIviate Airline Assistant\nType, speak, or upload your query. Hear answers, get translated replies and destination images!<br>Try: <i>'Book me a ticket to Paris for tomorrow as Rishikesh.'</i>", elem_id="top_mkd")

    with gr.Row():
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(label="Assistant", show_label=True, type="messages")
            text_input = gr.Textbox(label="Ask a question or request:", lines=1)
        with gr.Column(scale=1):
            gr.Markdown("#### Translation (Spanish):")
            translation_box = gr.Textbox(label="En español", interactive=False, lines=2)
            image_output = gr.Image(label="Related Vacation Image", interactive=False)
            audio_output = gr.Audio(label="Assistant Audio", interactive=False, type="filepath", show_label=True, autoplay=True)

    with gr.Row():
        audio_in = gr.Audio(label="Speak your request", type="filepath", show_label=True)
        clear_btn = gr.Button("Clear Conversation")

    def clear_hist():
        return [], "", "", None

    text_input.submit(
        full_agent, 
        inputs=[text_input, chatbot, audio_in],
        outputs=[chatbot, translation_box, audio_output, image_output],
        queue=True
    )

    audio_in.stop_recording(
        full_agent,
        inputs=[text_input, chatbot, audio_in],
        outputs=[chatbot, translation_box, audio_output, image_output]
    )

    clear_btn.click(clear_hist, None, [chatbot, translation_box, audio_output, image_output])

demo.queue()
demo.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
